In [ ]:
import pandas as pd
import os
import glob

# Папка с файлами
current_folder = os.getcwd()
print(f"Текущая папка: {current_folder}")

# Находим все CSV файлы
csv_files = glob.glob(os.path.join(current_folder, "*.csv"))

if not csv_files:
    print("Не найдено CSV файлов в текущей папке")
    exit()

print(f"Найдено файлов: {len(csv_files)}")

# Создаем папку для отфильтрованных файлов
filtered_folder = os.path.join(current_folder, "ru_country")
if not os.path.exists(filtered_folder):
    os.makedirs(filtered_folder)
    print(f"Создана папка для отфильтрованных файлов: {filtered_folder}")

# Варианты написания столбца country (разные регистры и варианты)
country_column_names = ['country', 'Country', 'COUNTRY', 'страна', 'Страна', 'СТРАНА']

processed_count = 0

for csv_file in csv_files:
    filename = os.path.basename(csv_file)
    print(f"\n{'='*60}")
    print(f"Обработка файла: {filename}")
    
    df = None
    used_encoding = None
    
    # Пробуем разные кодировки для чтения
    encodings = ['utf-8', 'utf-8-sig', 'windows-1251', 'cp1251']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(csv_file, encoding=encoding, low_memory=False)
            used_encoding = encoding
            print(f"  ✓ Прочитан с кодировкой: {encoding}")
            print(f"     Столбцов: {len(df.columns)}, Строк: {len(df)}")
            break
        except Exception as e:
            continue
    
    if df is None:
        print(f"  ✗ Не удалось прочитать файл")
        continue
    
    # Ищем столбец country
    country_col = None
    for col_name in country_column_names:
        if col_name in df.columns:
            country_col = col_name
            print(f"  ✓ Найден столбец: {country_col}")
            break
    
    if country_col is None:
        # Проверим все столбцы на наличие подстроки 'country'
        for col in df.columns:
            if isinstance(col, str) and 'country' in col.lower():
                country_col = col
                print(f"  ✓ Найден похожий столбец: {country_col}")
                break
    
    if country_col is None:
        print(f"  ⚠ Столбец 'country' не найден в файле")
        print(f"     Доступные столбцы: {list(df.columns)}")
        continue
    
    # Проверяем уникальные значения в столбце country
    unique_values = df[country_col].dropna().unique()
    print(f"  Уникальные значения в столбце {country_col}:")
    for val in unique_values[:10]:  # Показываем первые 10 значений
        print(f"    - '{val}' (тип: {type(val).__name__})")
    
    if len(unique_values) > 10:
        print(f"    ... и еще {len(unique_values) - 10} значений")
    
    # Фильтруем строки с значением RU
    # Создаем копию с нормализованными значениями для сравнения
    df_filtered = df.copy()
    
    # Приводим значения к строке и удаляем лишние пробелы
    df_filtered['country_normalized'] = df_filtered[country_col].astype(str).str.strip().str.upper()
    
    # Варианты написания RU
    ru_variants = ['RU', 'RUS', 'RUSSIA', 'РОССИЯ', 'РОССИЙСКАЯ ФЕДЕРАЦИЯ', 'RUSSIA FEDERATION', '643']
    
    # Фильтруем строки
    mask = df_filtered['country_normalized'].isin(ru_variants)
    
    # Также ищем частичное совпадение
    mask_partial = df_filtered['country_normalized'].str.contains('RU|РОСС', na=False, case=False)
    
    # Объединяем условия
    final_mask = mask | mask_partial
    
    # Применяем фильтр
    ru_df = df[final_mask].copy()
    
    print(f"  Исходное количество строк: {len(df)}")
    print(f"  Отфильтровано строк с RU: {len(ru_df)}")
    
    if len(ru_df) == 0:
        print(f"  ⚠ В файле нет строк со значением RU")
        continue
    
    # Удаляем вспомогательный столбец если он есть
    if 'country_normalized' in ru_df.columns:
        ru_df = ru_df.drop(columns=['country_normalized'])
    
    # Сохраняем отфильтрованный файл
    output_filename = f"RU_{filename}"
    output_path = os.path.join(filtered_folder, output_filename)
    
    try:
        # Сохраняем в UTF-8 с BOM для Excel
        ru_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"  ✓ Сохранен отфильтрованный файл: {output_filename}")
        
        # Дополнительная статистика
        print(f"  Статистика по отфильтрованным данным:")
        print(f"    - Столбцов: {len(ru_df.columns)}")
        print(f"    - Строк: {len(ru_df)}")
        
        # Проверяем распределение по странам в отфильтрованных данных
        if country_col in ru_df.columns:
            ru_countries = ru_df[country_col].value_counts().head(5)
            print(f"    - Распределение по странам (топ-5):")
            for country, count in ru_countries.items():
                percentage = count / len(ru_df) * 100
                print(f"      '{country}': {count} строк ({percentage:.1f}%)")
        
        processed_count += 1
        
    except Exception as e:
        print(f"  ✗ Ошибка при сохранении: {e}")

print(f"\n{'='*60}")
print(f"ОБРАБОТКА ЗАВЕРШЕНА!")
print(f"Обработано файлов: {processed_count} из {len(csv_files)}")
print(f"Отфильтрованные файлы сохранены в папке: {filtered_folder}")

# Сводная статистика по всем обработанным файлам
if processed_count > 0:
    print(f"\nСводная статистика по отфильтрованным файлам:")
    
    ru_files = glob.glob(os.path.join(filtered_folder, "RU_*.csv"))
    total_rows = 0
    total_files = len(ru_files)
    
    for ru_file in ru_files:
        try:
            df = pd.read_csv(ru_file, encoding='utf-8-sig', nrows=1)  # Читаем только заголовки
            with open(ru_file, 'r', encoding='utf-8-sig') as f:
                # Подсчитываем строки (минус заголовок)
                line_count = sum(1 for line in f) - 1
                total_rows += line_count
        except:
            continue
    
    print(f"  Всего файлов с данными RU: {total_files}")
    print(f"  Общее количество строк с RU: {total_rows:,}".replace(",", " "))
    
    # Показываем список созданных файлов
    print(f"\nСозданные файлы:")
    for i, file in enumerate(sorted(ru_files), 1):
        filename = os.path.basename(file)
        file_size = os.path.getsize(file) / 1024  # Размер в KB
        print(f"  {i:2}. {filename} ({file_size:.1f} KB)")